In [1]:
from desdeo_emo.EAs.IKRVEA import IK_RVEA
from desdeo_problem.surrogatemodels.SurrogateModels import GaussianProcessRegressor
from desdeo_problem import ExperimentalProblem
import sys 
sys.path.append("NumPy_path") 
import numpy as np
import pandas as pd

from sklearn.gaussian_process.kernels import Matern
from pymoo.factory import get_problem, get_reference_directions
import copy
from desdeo_tools.scalarization.ASF import SimpleASF

# we define the problem to be solved

In this example we have used the pymoo package to define dtlz2 problem with 3 objectives. Please not for now we cannot use the test_problem_maker class from DESDEO to use dtlz problems for online surrogate assisted methods and we need to define them manually.

In [2]:
def obj_function1(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,0]
def obj_function2(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,1]
def obj_function3(x):

    out = {
    "F": "",
    "G": "",
    }
    problem = get_problem("dtlz2", 12)
    problem._evaluate(x, out)
    return out['F'][:,2]

# Setting the parameters

In [3]:
refpoint = np.asarray([0.2,0.5,0.9])
n_obj = 3
n_var = n_obj + 9
var_names = ["x" + str(i + 1) for i in range(n_var)]
obj_names = ["f" + str(i + 1) for i in range(n_obj)]
unc_names = ["unc" + str(i + 1) for i in range(n_obj)]
# fundumentals of problem:

#creating the initial population
x = np.random.random((120, n_var))
initial_obj = {
    "F": "",
    "G": "",
    }
get_problem("dtlz2", 12)._evaluate(x, initial_obj)


Here we define the evolver and create the archive in pandas data frame type (datapd). Then we define the problem object.

## Note:
currently we are using the ExperimentalProblem class. In the future updates of DESDEO this name will change.

In [4]:
data = np.hstack((x, initial_obj['F']))
datapd = pd.DataFrame(data=data, columns=var_names+obj_names)

problem = ExperimentalProblem(data = datapd, objective_names=obj_names, variable_names=var_names,\
     uncertainity_names=unc_names, evaluators = [obj_function1, obj_function2, obj_function3])
problem.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})
u = 10 #number of solutions that we use to update surrogates in each iteration
evolver = IK_RVEA(
            problem, interact=True, n_iterations=1, n_gen_per_iter = 100,\
                 lattice_resolution=10, use_surrogates= True, population_size= 120, number_of_update=u)


## Training the surrogate models

In [5]:
problem.train(models=GaussianProcessRegressor, model_parameters={'kernel': Matern(nu=1.5)})


Defining the evolver with the surrogate models and some adjustments for interactive KRVEA.

The parameter u defines the number of solutions that need to be selected by the model management to evluate the original functions with them. 

Also in interactive K-RVEA we have a new conccept call interaction. A fixed number of iterations is called an interaction where we show the selected solutions to the DM and ask for a new reference point. 

# one interaction of IK-RVEA:

In [12]:
i = 0
while i<=5:
    pref, plot2 = copy.deepcopy(evolver.requests())
    pref[2].response = copy.deepcopy(pd.DataFrame([refpoint],\
        columns=pref[2].content['dimensions_data'].columns))
    plot= evolver.iterate(pref[2])   
    i+= 1

In [13]:
objectives = problem.archive.drop(problem.variable_names, axis=1).to_numpy()

We need to select the best solutions to be shown to the DM. Here we use Achievement Scalarizing function from DESDEO libarary. 

In [14]:

asf_values = SimpleASF([1]*problem.n_of_objectives).__call__(
    objectives, pref[2].response.values)
idx = np.argpartition(asf_values, u)[:u] #inddicies of best solutions based on ASF
Best_solutions = objectives[idx] 


In [15]:
refpoint

array([0.2, 0.5, 0.9])

In [16]:
Best_solutions

array([[0.19074031, 0.39382469, 0.90089551],
       [0.19549019, 0.44376983, 0.87739968],
       [0.19569516, 0.4436356 , 0.87820607],
       [0.1746375 , 0.48855943, 0.89025374],
       [0.14411824, 0.44857159, 0.88664468],
       [0.19729665, 0.44471376, 0.87477846],
       [0.18967828, 0.39375393, 0.90330336],
       [0.20389934, 0.4421285 , 0.88201424],
       [0.20401185, 0.49903816, 0.84373515],
       [0.20748028, 0.48109331, 0.87577967]])

# Visualization
Here after 10 iterations we need to select the best solutions to be shown to the DM.


In [17]:
import plotly.graph_objs as go
from pymoo.factory import get_problem, get_reference_directions, get_visualization
#the next three lines are to get the true pareto front
p = get_problem("dtlz2", 12)
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)
pf = p.pareto_front(ref_dirs)


x = Best_solutions[:,0]
y = Best_solutions[:,1]
z = Best_solutions[:,2]

trace1 = go.Scatter3d(x=x, y=y, z=z, mode="markers",)
trace2 = go.Scatter3d(x=[refpoint[0]], y=[refpoint[1]], z=[refpoint[2]], mode="markers")
trace3 = go.Mesh3d(x=pf[:,0], y=pf[:,1], z=pf[:,2])
fig = go.Figure(data = [trace1, trace2, trace3])
fig.show()

# Results
The green mesh is the true Pareto front, the blue dots are the solutions that Interactive K-RVEA has found, and the red dot is the reference point. 

Please note that we did not calculate the nondominated solutions in the archive for this example. In general, it is best if we perform the ASF function (or any other indicator) on nondominated solutions